## Добавление частотного смещения

Чтобы сделать наш моделируемый сигнал более реалистичным, мы внесем частотную рассинхронизацию. Предположим, что наша частота дискретизации составляет 1 МГц (на самом деле не важно, чему равна частота дискретизации, но вы увидите, почему мы выбрали именно такое значение). Если мы хотим смоделировать смещение частоты на 13 кГц (какое-то произвольное число), мы можем сделать это с помощью следующего кода:

In [ ]:
# применение частотного рассогласования к сигналу
fs = 1e6 # примем частоту дискретизации равной 1 МГц
fo = 13000 # simulate freq offset
Ts = 1/fs # период дискретизации
t = np.arange(0, Ts*len(samples), Ts)
samples = samples * np.exp(1j*2*np.pi*fo*t) # применяем частотное рассогласование

Ниже показан сигнал до и после частотной рассинхронизации:

# <КАРТИНКА>

Мы не строили график части Q, так как мы передавали BPSK и часть Q всегда была равна нулю. Теперь же, когда мы добавили частотный сдвиг, имитируя беспроводной канал, энергия распределяется по I и Q. С этого момента мы должны отображать как I, так и Q. Попробуйте в своем коде применить любое другое частотное смещение. Если вы уменьшите смещение примерно до 1 кГц, вы заметите, что огибающая сигнала похожа на синусоиду, так как она колеблется достаточно медленно, охватывая по несколько символов сразу.

Что касается выбора произвольной частоты дискретизации, если вы внимательно изучите код, вы заметите, что имеет значение отношение `fo` к `fs`.

Вы можете представить, что два блока кода, представленные ранее, имитируют беспроводной канал. Код должен идти после кода для передатчика (как мы делали в главе о формировании импульсов) и перед кодом приемника, что мы и рассмотрим в оставшейся части этой главы.

## Временная синхронизация

Когда мы передаем сигнал по беспроводной сети, он достигает приемника со случайным фазовым сдвигом из-за времени, необходимого для распространения сигнала. Мы не можем просто начать считывать отсчеты символов с нашей символьной скоростью, потому что мы вряд ли будем производить выборку в нужном месте импульса, как обсуждалось в конце главы «[Формирование импульса](https://pysdr.org/content/pulse_shaping.html)». Изучите три рисунка в конце этой главы, если вы не понимаете.

Большинство методов временной синхронизации использую принцип работы контура фазовой [автоподстройки частоты](https://ru.wikipedia.org/wiki/%D0%A4%D0%B0%D0%B7%D0%BE%D0%B2%D0%B0%D1%8F_%D0%B0%D0%B2%D1%82%D0%BE%D0%BF%D0%BE%D0%B4%D1%81%D1%82%D1%80%D0%BE%D0%B9%D0%BA%D0%B0_%D1%87%D0%B0%D1%81%D1%82%D0%BE%D1%82%D1%8B) (ФАПЧ или PLL); мы не будем изучать здесь PLL, но важно знать этот термин, и хотя бы в общих чертах понимать принцип работы. PLL — это системы с обратной связью, которые используют обратную связь для постоянной регулировки чего-либо; в нашем случае временной сдвиг позволяет нам производить выборку на пике цифровых символов.

Вы можете представить восстановление синхронизации как блок в приемнике, который принимает один поток отсчетов и выводит другой поток отсчетов (аналогично фильтру). Мы программируем этот блок с информацией о нашем сигнале, наиболее важной из которых является количество отсчетов на символ (или устанавливаем предположение о значении этой величины, если мы не уверены на 100%, что было передано). Этот блок действует как «дециматор», т. е. размер выборки на выходе будет частью размера входной выборки. Нам нужен один отсчет на цифровой символ, поэтому величина децимации просто будет равна количеству отсчетов на символ. Если передатчик передает со скоростью 1 млн символов в секунду, а мы делаем выборку со скоростью 16 млн выборок в секунду, мы получим 16 отсчетов на символ. Это будет частота дискретизации, поступающая в блок синхронизации времени. Частота дискретизации сигнала на выходе блока будет равна 1 Мвыб/с, потому что нам нужен один отсчет на цифровой символ.

Большинство методов временной синхронизации основаны на том факте, что наши цифровые символы "растут", а затем "падают", а вершина — это точка, в которой мы хотим произвести отсчет символа. Другими словами, мы выбираем максимальную точку после получения абсолютного значения:

# <КАРТИНКА>

Существует множество методов временной синхронизации, очень похожих на PLL. Как правило, разница между ними заключается в уравнении, используемом для выполнения «коррекции» временного смещения, которое мы обозначаем как $\mu$ или `mu` в коде. Значение `mu` выражается в единицах сэмплов м обновляется при каждой итерации цикла. Вы можете думать об этом как "_на сколько отсчетов мы должны сместиться, чтобы иметь возможность произвести отсчет в «идеальное» время_". Таким образом, если `mu = 3,61`, это означает, что мы должны сдвинуть входные данные на 3,61 отсчета, чтобы сэмплировать в нужном месте. Поскольку у нас 8 отсчетов на символ, если `mu` превысит 8, то `mu` просто вернется к нулю.

Следующий код Python реализует синхронизацию согласно [алгоритму Миллера и Мюллера](https://wirelesspi.com/mueller-and-muller-timing-synchronization-algorithm/).

In [1]:
mu = 0 # исходная оценка фазы символа
out = np.zeros(len(samples) + 10, dtype=np.complex)
out_rail = np.zeros(len(samples) + 10, dtype=np.complex) # хранилище значений отсчетов; на каждой итерации нам необходимы два предыдущих значения плюс текущее
i_in = 0 # индекс входных отсчетов
i_out = 2 # индекс выходных отсчетов (пусть первые два отсчета будут нулями)
while i_out < len(samples) and i_in+16 < len(samples):
    out[i_out] = samples[i_in + int(mu)] # берем "лучший" отсчет
    out_rail[i_out] = int(np.real(out[i_out]) > 0) + 1j*int(np.imag(out[i_out]) > 0)
    x = (out_rail[i_out] - out_rail[i_out-2]) * np.conj(out[i_out-1])
    y = (out[i_out] - out[i_out-2]) * np.conj(out_rail[i_out-1])
    mm_val = np.real(y - x)
    mu += sps + 0.3*mm_val
    i_in += int(np.floor(mu)) # округляем вниз до целого значения, чтобы использовать его как индекс
    mu = mu - np.floor(mu) # избавляемся от целочисленной части
    i_out += 1 # инкрементируем индекс выходных отсчетов
out = out[2:i_out] # отрезаем первые два отсчета и все те, что идут после i_out
samples = out # включите эту строку только в том случае, если вы хотите соединить этот фрагмент кода с петлей Костаса позже

NameError: name 'np' is not defined

В блок восстановления синхронизации подаются «принятые» отсчеты, и он генерирует выходные отсчеты по одному за раз (обратите внимание, что i_out увеличивается на 1 на каждой итерации цикла). Блок восстановления не выкидывает просто «принятые» отсчеты один за другим на выход благодаря тому, как в цикле изменяется i_in. Он будет пропускать некоторые сэмплы в попытке получить «правильный», который будет соответствовать пику импульса. Когда цикл обрабатывает сэмплы, он медленно синхронизируется с символом или, по крайней мере, пытается это сделать, настраивая `mu`. Согласно коду, целая часть `mu` добавляется к `i_in`, а затем вычитается из `mu` (имейте в виду, что `mm_val` может принимать как отрицательные так и положительным значения на каждой итерации цикла). После полной синхронизации цикл должен осуществлять выборку только в центре каждого символа/импульса. Вы можете настроить константу `0,3`, которая изменит чувствительность контура обратной связи; более высокое значение повысит чувствительность контура, но может привести к полной рассинхронизации.

На следующем графике показан пример вывода синхронизатора, в котором мы отключили дробную временную задержку, а также смещение частоты. Мы показываем только I, потому что Q состоит из нулей с отключенным смещением частоты. Три графика наложены друг на друга, чтобы показать, как биты выравниваются по вертикали.

### Верхний график

Исходные символы BPSK, т. е. 1 и -1. Напомним, что между ними есть нули, потому что мы считываем по 8 отсчетов на символ.

### Средний график

Отсчеты после формирования импульса, но до синхронизатора.

### Нижний график

Вывод символьного синхронизатора, содержащий по 1 отсчету на символ. То есть эти отсчеты можно подавать непосредственно в демодулятор.

# <КАРТИНКА>

Давайте сосредоточимся на нижнем графике, который является выходом синхронизатора. Потребовалось около 30 символов для того, чтобы найти правильную задержку и установилась синхронизация. Из-за того, что на синхронизацию всегда требуется какое-то время, многие протоколы связи предусматривают использование преамбулы, содержащей синхропоследовательность: она сообщает синхронизатору о начале нового пакета и дает приемнику время для синхронизации с ним. Но после этих ~30 сэмплов синхронизатор работает отлично. Мы получили отчетливые 1 и -1, соответствующие входным данным. Также не забывайте, что любой шум замедлит процесс синхронизации и тогда нам потребуется больше отсчетов. Попробуйте добавить в свой код шум или временные сдвиги и посмотрите, как в таком случае будет работать синхронизатор. Если бы мы использовали QPSK, то имели бы дело с комплексными числами, но подход был бы таким же.